In [153]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import re
import time
import random
import datetime
import pandas as pd

In [177]:
article_title = [] # 文章標題
article_URL = [] # 文章內文連結
article_author = [] # 文章作者
article_date = [] # 發文日期
article_author_ip = [] # 發文者IP
article_messages = [] #文章中的每筆留言資訊 (含留言者ID、時間、推噓中立狀態、內容)
article_message_count_all = [] # 整體留言數量
article_message_count_status = [] # 推文-噓文數量，顯示何者狀態較多
article_message_count_push = [] # 推文數量
article_message_count_boo = []  # 噓文數量
article_message_count_neutral = [] # 中立數量


In [178]:
url = 'https://www.ptt.cc/bbs/HatePolitics/index.html'
resp = requests.get(url, cookies={'over18': '1'})
soup = BeautifulSoup(resp.text, 'html.parser')
main_list = soup.find('div', class_='bbs-screen')

count = 0
while count < 10:
    # 依序檢查文章列表中的 tag, 遇到分隔線就結束，忽略這之後的文章
    
    for div in main_list.findChildren('div', recursive=False):    
        if count < 10:
            class_name = div.attrs['class']
            # 遇到目標文章
            if class_name and 'r-ent' in class_name:
                div_title = div.find('div', class_='title')
                a_title = div_title.find('a', href=True)
                # 如果文章已經被刪除則跳過
                if not a_title or not a_title.has_attr('href'):
                    continue
    
                article_URL_text = urljoin('https://www.ptt.cc', a_title['href'])
                article_URL.append(article_URL_text)
                article_title_text = a_title.text
                print(article_title_text)
                
                
                article_content_response =  requests.get(article_URL_text, cookies={'over18': '1'})
                #print(article_content_response.text)
                soup = BeautifulSoup(article_content_response.text)
                # 取得文章內容主體
                main_content = soup.find(id='main-content')
                # 假如文章有屬性資料 (meta), 我們在從屬性的區塊中爬出作者 (author), 文章標題 (title), 發文日期 (date)
                metas = main_content.select('div.article-metaline')
                author = ''
                title = ''
                date = ''
                if metas:
                    if metas[0].select('span.article-meta-value')[0]:
                        author = metas[0].select('span.article-meta-value')[0].string.split('(')[0]
                        article_author.append(author)
                        
                    if metas[1].select('span.article-meta-value')[0]:
                        title = metas[1].select('span.article-meta-value')[0].string
                        article_title.append(title)
                    if metas[2].select('span.article-meta-value')[0]:
                        date = metas[2].select('span.article-meta-value')[0].string
                        date = datetime.datetime.strptime(date, '%a %b %d %H:%M:%S %Y').strftime('%Y/%m/%d %H:%M:%S')
                        article_date.append(date)
                    # 從 main_content 中移除 meta 資訊（author, title, date 與其他看板資訊）
                    for m in metas:
                        m.extract()
                    for m in main_content.select('div.article-metaline-right'):
                        m.extract()
                # 取得留言區主體
                pushes = main_content.find_all('div', class_='push')
                for p in pushes:
                    p.extract()
                # 假如文章中有包含「※ 發信站: 批踢踢實業坊(ptt.cc), 來自: xxx.xxx.xxx.xxx」的樣式
                # 透過 regular expression 取得 IP
                try:
                    ip = main_content.find(text=re.compile(u'※ 發信站:'))
                    ip = re.search('[0-9]*\.[0-9]*\.[0-9]*\.[0-9]*', ip).group()
                    article_author_ip.append(ip)
                except Exception as e:
                    ip = ''
                    article_author_ip.append(ip)
                # 移除文章主體中 '※ 發信站:', '◆ From:', 空行及多餘空白 (※ = u'\u203b', ◆ = u'\u25c6')
                # 保留英數字, 中文及中文標點, 網址, 部分特殊符號
                filtered = []
                for v in main_content.stripped_strings:
                    # 假如字串開頭不是特殊符號或是以 '--' 開頭的, 我們都保留其文字
                    if v[0] not in [u'※', u'◆'] and v[:2] not in [u'--']:
                        filtered.append(v)
                # 定義一些特殊符號與全形符號的過濾器
                expr = re.compile(u'[^一-龥。；，：“”（）、？《》\s\w:/-_.?~%()]')
                for i in range(len(filtered)):
                    filtered[i] = re.sub(expr, '', filtered[i])
                # 移除空白字串, 組合過濾後的文字即為文章本文 (content)
                filtered = [i for i in filtered if i]
                content = ' '.join(filtered)
                
                # 處理留言區
                # p 計算推文數量
                # b 計算噓文數量
                # n 計算箭頭數量
                p, b, n = 0, 0, 0
                messages = []
                for push in pushes:
                    # 假如留言段落沒有 push-tag 就跳過
                    if not push.find('span', 'push-tag'):
                        continue
                    # 過濾額外空白與換行符號
                    # push_tag 判斷是推文, 箭頭還是噓文
                    # push_userid 判斷留言的人是誰
                    # push_content 判斷留言內容
                    # push_ipdatetime 判斷留言日期時間
                    push_tag = push.find('span', 'push-tag').string.strip(' \t\n\r')
                    push_userid = push.find('span', 'push-userid').string.strip(' \t\n\r')
                    push_content = push.find('span', 'push-content').strings
                    push_content = ' '.join(push_content)[1:].strip(' \t\n\r')
                    push_ipdatetime = push.find('span', 'push-ipdatetime').string.strip(' \t\n\r')
                    
                    # 整理打包留言的資訊, 並統計推噓文數量
                    
                    messages.append({
                        'push_tag': push_tag,
                        'push_userid': push_userid,
                        'push_content': push_content,
                        'push_ipdatetime': push_ipdatetime})    
                    if push_tag == u'推':
                        p += 1
                    elif push_tag == u'噓':
                        b += 1
                    else:
                        n += 1
                        
                    # 統計推噓文
                    # count 為推噓文相抵看這篇文章推文還是噓文比較多
                    # all 為總共留言數量 
                    #message_count = {'all': p+b+n, 'count': p-b, 'push': p, 'boo': b, 'neutral': n}
                article_message_count_all.append(p+b+n)
                article_message_count_status.append(p-b)
                article_message_count_push.append(p)
                article_message_count_boo.append(b)
                article_message_count_neutral.append(n)
                
                # 檢視文章中是否有留言，沒有的話，以'no comments' 作為值
                if len(messages) == 0 :
                    messages.append('no comments')
                    article_messages.append(messages)
                else:
                    article_messages.append(messages)
                count += 1
                print('--------------------------------------------------------------------')
        else:
            break
    
            
        if class_name and 'r-list-sep' in class_name:
            print('Reach the last article in the first page, go to the next page')
            break
           
    print('******************************************************************************')
    print('Reach the last article,go to the next page')
    print('******************************************************************************')
    soup = BeautifulSoup(resp.text, 'html.parser')   
    next_url = 'https://www.ptt.cc' + soup.select_one('#action-bar-container > div > div.btn-group.btn-group-paging > a:nth-child(2)')['href']
    resp = requests.get(next_url, cookies={'over18': '1'})
    soup = BeautifulSoup(resp.text, 'html.parser')
    main_list = soup.find('div', class_='bbs-screen')
    time.sleep(random.uniform(1, 3))
    
else:
    print(len(article_title))

[討論] 台灣這麼小 找不到顏清標？？？
--------------------------------------------------------------------
Re: [新聞] 快訊／補選前夕　顏清標驚傳昏倒送急診
--------------------------------------------------------------------
[討論] 為什麼法西斯的藝術感都這麼差
--------------------------------------------------------------------
Re: [新聞] 快訊／補選前夕　顏清標驚傳昏倒送急診
--------------------------------------------------------------------
[黑特] 你們還有沒有人性啊！！！
--------------------------------------------------------------------
[討論] 484要肉搜是哪家記者氣病顏清標？
--------------------------------------------------------------------
[討論] 假設柯文哲站台挺顏 有關鍵影響？
--------------------------------------------------------------------
Re: [黑特] 塔綠班打顏打到無下限了
--------------------------------------------------------------------
Re: [新聞] 快訊／補選前夕　顏清標驚傳昏倒送急診
--------------------------------------------------------------------
Re: [新聞] 選戰倒數！被問顏母墓地合法嗎？ 顏清標
--------------------------------------------------------------------
************************************************************************

In [179]:
df = pd.DataFrame({"article_date":article_date,"article_title":article_title,"article_author":article_author,"article_author_ip":article_author_ip,\
                  "article_URL":article_URL,"article_message_count_all":article_message_count_all,"article_message_count_status":article_message_count_status,\
                  "article_message_count_push":article_message_count_push,"article_message_count_boo":article_message_count_boo,"article_message_count_neutral":article_message_count_neutral,\
                  "article_messages":article_messages})

In [180]:
df


,article_date,article_title,article_author,article_author_ip,article_URL,article_message_count_all,article_message_count_status,article_message_count_push,article_message_count_boo,article_message_count_neutral,article_messages
0,2022/01/06 23:02:38,[討論] 台灣這麼小 找不到顏清標？？？,linda17a3,114.47.96.69,https://www.ptt.cc/bbs/HatePolitics/M.16414813...,8,2,2,0,6,"[{'push_tag': '→', 'push_userid': 'kuninaka', ..."
1,2022/01/06 23:04:37,Re: [新聞] 快訊／補選前夕 顏清標驚傳昏倒送急診,ysb727,101.136.185.133,https://www.ptt.cc/bbs/HatePolitics/M.16414814...,7,2,2,0,5,"[{'push_tag': '→', 'push_userid': 'h100733', '..."
2,2022/01/06 23:10:30,[討論] 為什麼法西斯的藝術感都這麼差,sanae0307,114.136.140.217,https://www.ptt.cc/bbs/HatePolitics/M.16414818...,6,2,3,1,2,"[{'push_tag': '推', 'push_userid': 'kuninaka', ..."
3,2022/01/06 23:10:34,Re: [新聞] 快訊／補選前夕 顏清標驚傳昏倒送急診,bluenan,114.34.118.117,https://www.ptt.cc/bbs/HatePolitics/M.16414818...,20,1,2,1,17,"[{'push_tag': '→', 'push_userid': 'kterry01', ..."
4,2022/01/06 23:12:31,[黑特] 你們還有沒有人性啊！！！,tn00270144,27.240.250.127,https://www.ptt.cc/bbs/HatePolitics/M.16414819...,43,6,14,8,21,"[{'push_tag': '→', 'push_userid': 'brella', 'p..."
5,2022/01/06 23:19:07,[討論] 484要肉搜是哪家記者氣病顏清標？,ysb727,101.136.185.133,https://www.ptt.cc/bbs/HatePolitics/M.16414823...,41,19,20,1,20,"[{'push_tag': '推', 'push_userid': 'kuninaka', ..."
6,2022/01/06 23:22:49,[討論] 假設柯文哲站台挺顏 有關鍵影響？,chungrew,1.168.101.135,https://www.ptt.cc/bbs/HatePolitics/M.16414825...,12,5,5,0,7,"[{'push_tag': '→', 'push_userid': 'kuninaka', ..."
7,2022/01/06 23:24:41,Re: [黑特] 塔綠班打顏打到無下限了,outsider666,101.136.36.182,https://www.ptt.cc/bbs/HatePolitics/M.16414826...,13,-3,2,5,6,"[{'push_tag': '推', 'push_userid': 'kuninaka', ..."
8,2022/01/06 23:25:19,Re: [新聞] 快訊／補選前夕 顏清標驚傳昏倒送急診,XXXXGAY,180.177.41.196,https://www.ptt.cc/bbs/HatePolitics/M.16414827...,48,19,19,0,29,"[{'push_tag': '→', 'push_userid': 'dog990999',..."
9,2022/01/06 23:25:39,Re: [新聞] 選戰倒數！被問顏母墓地合法嗎？ 顏清標,krajicek,114.27.60.223,https://www.ptt.cc/bbs/HatePolitics/M.16414827...,6,4,4,0,2,"[{'push_tag': '→', 'push_userid': 'sumarai', '..."


### 測試區